# Benchmarks and Tests of Maps for Single-Element Substrates with a Defect

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
default_fontsize = plt.rcParams['font.size']
publication_fontsize_large = 20
publication = False
if publication: plt.rcParams.update({'font.size': publication_fontsize_large})

## Pt(111) Surface

Generate the ASE.Atoms instance using the fcc111 build function. The atom at the center of the cell on the top face is index number 42.

In [ ]:
from ase.build import fcc111
Pt111 = fcc111("Pt",size=(4,4,3),a=3.94,orthogonal=True,periodic=True,vacuum = 10)
del(Pt111[42])

From the `ASE.Cell` and the generated `ASE.Atoms`, create a `MapSy.Grid` and a `MapSy.System`

In [ ]:
from mapsy.data import Grid
grid: Grid = Grid(cell=Pt111.cell)

In [ ]:
from mapsy.data import System
system: System = System(grid, Pt111, dimension=2, axis=2)

In [ ]:
from mapsy.io.parser import ContactSpaceGenerator, ContactSpaceModel
contactspacesettings = ContactSpaceModel.parse_obj({"mode": "system", "distance": 3.5, "spread": 1.0, "cutoff": 80, "threshold": -1, "side":1})
contactspace = ContactSpaceGenerator(contactspacesettings).generate(system)

In [ ]:
contactspace.data

In [ ]:
from mapsy.symfunc.input import SymmetryFunctionsModel, SymFuncModel
from mapsy.symfunc.parser import SymmetryFunctionsParser
symfuncsettings = SymmetryFunctionsModel.parse_obj({"functions": [SymFuncModel.parse_obj({"type":"ac","radius":4.5,"order":10,"compositional":False,"structural":True}),SymFuncModel.parse_obj({"type":"ac","radius":4.5,"order":10,"compositional":False,"structural":True,"radial":False})]})
symmetryfunctions = SymmetryFunctionsParser(symfuncsettings).parse()

In [ ]:
from mapsy.maps import Maps
maps = Maps(system,symmetryfunctions,contactspace)

In [ ]:
data = maps.atcontactspace()

We can visualize features to check how they look using `Maps.plot(feature: str)` or `Maps.plot(index: int)`. NOTE: to get the top face of the slab, we need to select `region=1`. 

In [ ]:
fig, axes = maps.plot(index=0, cmap='Spectral', set_aspect='scaled', levels=20)
plt.show()

## Dimensionality Reduction (PCA)

For visualization and post-processing purposes, perform dimensionality reduction on the generated features. Three components are useful for 2D and 3D plots.

In [ ]:
fig, ax1, ax2 = maps.reduce(scale=True)
if (publication) : 
    ax1.set_title('PCA')
    fig.tight_layout()

In [ ]:
npca = 6
maps.reduce(npca, scale=True)

We can visually inspect how the PCAs correlate with the Cartesian coordinates of the points (e.g., PC0 still distinguishing between atop positions, while PC1 correlated with the distance from the defect).

In [ ]:
for i in range(npca):
    fig, axes = maps.plot(feature=f'pca{i}', axes=['x', 'y'],cmap='Spectral', set_aspect='scaled', levels=20)
    axes.set_title(f'PCA {i+1}')
    axes.set_xlabel('x (Å)')
    axes.set_ylabel('y (Å)')
    plt.show()

In [ ]:
fig, gs = maps.scatter_pca_grid(index=0,cmap='Spectral',set_aspect='equal',s=70, alpha=0.05)
fig.tight_layout()

We can also verify how the contact space is transformed (folded) in the symmetry function space. 

In [ ]:
fig, ax = maps.scatter(index=0, cmap='Spectral', axes=['x','y'], alpha=1., set_aspect='scaled', s=10)
ax.set_xlabel('x (Å)')
ax.set_ylabel('y (Å)')
if publication : ax.set_title("")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4*1))
fig.subplots_adjust(hspace=0.3)
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_title("")
x1m = maps.data['pca0'].values.astype(np.float64)
x2m = maps.data['pca1'].values.astype(np.float64)
fm = maps.data[maps.features[0]].values.astype(np.float64)
fmin = np.min(fm)
fmax = np.max(fm)
scatter = ax.scatter(x1m,x2m,c=fm,vmin=fmin,vmax=fmax,cmap='Spectral',alpha=0.2,s=60,edgecolors='black')
ax.axis('on')
plt.show()

## Perform Clustering on Generated Features

Use SpectralClustering to find N clusters in the featured data. 

In [ ]:
ntries = 1
if publication: ntries = 100
fig, ax1, ax2 = maps.cluster(maxclusters=20, ntries=ntries)
if publication: 
    ax1.set_title('')
    ax2.set_title('')
plt.show()

In [ ]:
maps.cluster(nclusters=18)

Given the clusters, plot the connectivity matrix

In [ ]:
plt.matshow(maps.cluster_edges)

Given the clusters and the connectivity, find the high-symmetry sites

In [ ]:
maps.sites()

Visualize the results

In [ ]:
fig, ax = maps.scatter(feature='Cluster', categorical=True, alpha=0.8, s=20, set_aspect='scaled', centroids=True)
ax.set_xlabel('x (Å)')
ax.set_ylabel('y (Å)')
ax.set_title('Clusters')
if publication:
    ax.set_title('')
    ax.get_legend().remove()
plt.show()

In [ ]:
axes = ['pca0','pca1']
fig, ax = maps.scatter(feature='Cluster', categorical=True, axes=axes, alpha=0.2, s=70, edgecolors='black', set_aspect='on')
G = nx.from_numpy_array(maps.cluster_edges,create_using=nx.DiGraph,parallel_edges=False)
pos = maps.data.loc[maps.centroids,axes].values
weights = [ d['weight']/200 for (u, v, d) in G.edges(data=True)]
nx.draw(G, pos, node_size=maps.cluster_sizes, width=weights, ax=ax, alpha=0.5, edgecolors='black')
limits=ax.axis('on') # turns on axis
ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_title('Clusters')
if publication:
    ax.set_title('')
    ax.get_legend().remove()
plt.show()